## Building a Data Lake on AWS

In this workshop we are going to be using Github activiity from the [Github Archive](https://www.githubarchive.org/). We will start by uploading a sample of the Github Archive to S3 and register the raw data with the Glue Data Catalog. Once we have the data registered we will transform the data to get only the columns necessary to run NLP on the comments of the commits in the Github history. We will be using [Amazon Comprehend](https://aws.amazon.com/comprehend/) to get the sentiment of the comments as well as determine 

Topics:

* Ingestion - Kinesis? Taxi DataSet? Something Else?
* Store and Catalog - S3 and Glue - Fine Grianed Access? JSON, CSV, Parquet?
* Transform - Glue ETL
* Visualize - QuickSight? Bokeh? matplotlib?

In [102]:
import boto3
import botocore
import json
import time
import os

import project_path # path to helper methods
from lib import workshop

glue = boto3.client('glue')

session = boto3.session.Session()
region = session.region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')

bucket = '{{s3 workshop bucket}}' # Bucket for all data used in this workshop

In [ ]:
from datetime import datetime, timedelta

yesterday = datetime.now() - timedelta(days=1)
timestamp = yesterday.strftime('%Y-%m-%d')
print(timestamp)

### Download Github Activity 

We will download an hour from yesterday of Github Activity. You could easily expand to downloading larger sets of Github Activity and the steps below would still apply.

In [ ]:
!wget http://data.gharchive.org/{timestamp}-{0..23}.json.gz

### Unzip Github Activity archive

In [ ]:
!gunzip {timestamp}*.json.gz
!ls

### View raw json file

In [ ]:
!head {timestamp}-4.json

### [Upload to S3](https://docs.aws.amazon.com/AmazonS3/latest/dev/Welcome.html)

Next, we will upload the json file created above to S3 to be used later in the workshop.

[s3.upload_file](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.upload_file) boto3 documentation

In [ ]:
file_name = timestamp+'-4.json'
session.resource('s3').Bucket(bucket).Object(os.path.join('github', 'raw', file_name)).upload_file(file_name)

### Discover the data in your Data Lake

In this next section we will be using [AWS Glue](https://aws.amazon.com/glue/) to discover, catalog, and transform your data.Glue currently only supports `Python 2.7`, hence we'll write the script in `Python 2.7`.

### Permission setup for invoking AWS Glue from this Notebook
In order to enable this Notebook to run AWS Glue jobs, we need to add one additional permission to the default execution role of this notebook. We will be using SageMaker Python SDK to retrieve the default execution role and then you have to go to [IAM Dashboard](https://console.aws.amazon.com/iam/home) to edit the Role to add AWS Glue specific permission. 

### Finding out the current execution role of the Notebook
We are using SageMaker Python SDK to retrieve the current role for this Notebook which needs to be enhanced.

In [ ]:
# Import SageMaker Python SDK to get the Session and execution_role
import sagemaker
from sagemaker import get_execution_role
sess = sagemaker.Session()
role = get_execution_role()
role_name = role[role.rfind('/') + 1:]
print(role_name)

### Adding AWS Glue as an additional trusted entity to this role
This step is needed if you want to pass the execution role of this Notebook while calling Glue APIs as well without creating an additional **Role**. If you have not used AWS Glue before, then this step is mandatory. 

If you have used AWS Glue previously, then you should have an already existing role that can be used to invoke Glue APIs. In that case, you can pass that role while calling Glue (later in this notebook) and skip this next step.

On the IAM dashboard, please click on **Roles** on the left sidenav and search for this Role. Once the Role appears, click on the Role to go to its **Summary** page. Click on the **Trust relationships** tab on the **Summary** page to add AWS Glue as an additional trusted entity. 

Click on **Edit trust relationship** and replace the JSON with this JSON.
```
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": [
          "sagemaker.amazonaws.com",
          "glue.amazonaws.com"
        ]
      },
      "Action": "sts:AssumeRole"
    }
  ]
}
```
Once this is complete, click on **Update Trust Policy** and you are done.

In [ ]:
print("https://console.aws.amazon.com/iam/home?region={0}#/roles/{1}".format(region, role_name))

### Create [Glue Catalog Database](https://docs.aws.amazon.com/glue/latest/dg/define-database.html)

When you define a table in the AWS Glue Data Catalog, you add it to a database. A database is used to organize tables in AWS Glue. You can organize your tables using a crawler or using the AWS Glue console. A table can be in only one database at a time.

There is a central Glue Catalog for each AWS account. When creating the database you will use your account id declared above as `account_id`

[glue.create_database](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/glue.html#Glue.Client.create_database)

In [ ]:
database_name = 'github'

response = glue.create_database(
    CatalogId=account_id,
    DatabaseInput={
        'Name': database_name,
        'Description': 'Database for Github Activity'
    }
)

### Use a [Glue Crawler](https://docs.aws.amazon.com/glue/latest/dg/add-crawler.html) to Discover the raw data

You can use a crawler to populate the AWS Glue Data Catalog with tables. This is the primary method used by most AWS Glue users. You add a crawler within your Data Catalog to traverse your data stores. The output of the crawler consists of one or more metadata tables that are defined in your Data Catalog. Extract, transform, and load (ETL) jobs that you define in AWS Glue use these metadata tables as sources and targets.

[What Data Stores can I crawl?](https://docs.aws.amazon.com/glue/latest/dg/add-crawler.html#crawler-data-stores)

[glue.create_crawler](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/glue.html#Glue.Client.create_crawler)

In [ ]:
raw_crawler_name = 'github_raw'
raw_crawler_path = 's3://'+bucket+'/github/raw/'

response = glue.create_crawler(
    Name=raw_crawler_name,
    Role=role,
    DatabaseName=database_name,
    Description='Crawler for the raw Github Activity',
    Targets={
        'S3Targets': [
            {
                'Path': raw_crawler_path
            }
        ]
    },
    SchemaChangePolicy={
        'UpdateBehavior': 'UPDATE_IN_DATABASE',
        'DeleteBehavior': 'DEPRECATE_IN_DATABASE'
    }
)

### Start the Glue Crawler

Execute the cell below and browse to the Glue CRawler we created above. From there you will click the `Run Crawler` button to start the crawl of the raw Github activity data.

In [ ]:
print ("Raw Crawler: https://{0}.console.aws.amazon.com/glue/home?region={0}#crawler:name={1}".format(region, crawler_name))

### Checking Glue crawler status

In [ ]:
crawler_status = glue.get_crawler(Name=raw_crawler_name)['Crawler']['State']
while crawler_status not in ('READY'):
    crawler_status = glue.get_crawler(Name=raw_crawler_name)['Crawler']['State']
    print(crawler_status)
    time.sleep(30)

### View Github Activity Data 

To see the raw Github activity we will be installing a python library for querying the data in the Glue Data Catalog with Athena. More information about [PyAthena](https://pypi.org/project/PyAthena/)

In [ ]:
!pip install PyAthena

In [103]:
from pyathena import connect
from pyathena.util import as_pandas

cursor = connect(region_name=region, s3_staging_dir='s3://'+bucket+'/github/temp').cursor()

cursor.execute('select * from github.raw limit 10')

df = as_pandas(cursor)
df.head(10)

,id,type,actor,repo,payload,public,created_at,org
0,8783176689,PushEvent,"{id=1928756, login=alexwnovak, display_login=a...","{id=160251010, name=malecus/Blog, url=https://...","{push_id=3152989573, size=9, distinct_size=9, ...",True,2018-12-20T04:17:43Z,"{id=45400087, login=malecus, gravatar_id=, url..."
1,8783176696,WatchEvent,"{id=6754119, login=sivachamarthi, display_logi...","{id=72953548, name=DaniJG/DockNetFiddle, url=h...","{push_id=null, size=null, distinct_size=null, ...",True,2018-12-20T04:17:43Z,None
2,8783176698,PushEvent,"{id=32147718, login=vivekmallampati, display_l...","{id=162474698, name=vivekmallampati/My_Bhagava...","{push_id=3152989582, size=1, distinct_size=1, ...",True,2018-12-20T04:17:43Z,None
3,8783176701,PushEvent,"{id=16398401, login=shaoyudong, display_login=...","{id=160145754, name=shaoyudong/anu, url=https:...","{push_id=3152989584, size=1, distinct_size=1, ...",True,2018-12-20T04:17:43Z,None
4,8783176703,CreateEvent,"{id=39166683, login=zankner, display_login=zan...","{id=162525769, name=zankner/ml-a-z-, url=https...","{push_id=null, size=null, distinct_size=null, ...",True,2018-12-20T04:17:43Z,None
5,8783176714,PushEvent,"{id=8194039, login=abaga129, display_login=aba...","{id=90333237, name=abaga129/dplug, url=https:/...","{push_id=3152989587, size=1, distinct_size=1, ...",True,2018-12-20T04:17:43Z,None
6,8783176715,PushEvent,"{id=19721204, login=ejmiranda, display_login=e...","{id=162525406, name=ejmiranda/ruby, url=https:...","{push_id=3152989590, size=1, distinct_size=1, ...",True,2018-12-20T04:17:44Z,None
7,8783176717,WatchEvent,"{id=327432, login=mridang, display_login=mrida...","{id=62844767, name=lukechilds/zsh-nvm, url=htt...","{push_id=null, size=null, distinct_size=null, ...",True,2018-12-20T04:17:44Z,None
8,8783176725,PushEvent,"{id=42481851, login=pg132, display_login=pg132...","{id=156795038, name=pg132/universeCreatorIdle,...","{push_id=3152989599, size=1, distinct_size=1, ...",True,2018-12-20T04:17:44Z,None
9,8783176729,PushEvent,"{id=24628622, login=xiaojiew1, display_login=x...","{id=156340066, name=xiaojiew1/DRREC, url=https...","{push_id=3152989601, size=1, distinct_size=1, ...",True,2018-12-20T04:17:44Z,None


## Transform Raw data to provide insights and visualization

### Uploading the code and other dependencies to S3 for AWS Glue
In order to run your code in AWS Glue, we need to upload the code and dependencies directly to S3 and pass those locations while invoking the Glue job.

### Transform Github Activity Data

We will create a Pyspark job to filter down the raw data to only JSON documents with type `PushEvent` to get the commit messages and run them through [Amazon Comprehend](https://aws.amazon.com/comprehend/)

In [ ]:
%%writefile github_etl.py

import os
import sys
import boto3

from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.dynamicframe import DynamicFrame

import pyspark.sql.functions as F
from pyspark.sql import Row, Window, SparkSession
from pyspark.sql.types import *
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql.functions import *

args = getResolvedOptions(sys.argv, ['JOB_NAME', 'S3_OUTPUT_BUCKET', 'S3_OUTPUT_KEY_PREFIX'])

sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args['JOB_NAME'], args)

github = glueContext.create_dynamic_frame.from_catalog(database="github", table_name="raw")

githubDF = github.toDF()
githubCommitsDF = githubDF.select("id", "created_at", "type", explode("payload.commits").alias("commits"), "repo", "actor")\
                    .select("id", "commits.author.name", "commits.message", "repo.url", "actor.login", month(githubDF.created_at).alias('dt_month'), dayofmonth(githubDF.created_at).alias('dt_day'), year(githubDF.created_at).alias('dt_year'), hour(githubDF.created_at).alias('dt_hour'))\
                    .where(githubDF.type == "PushEvent")

ghsink = DynamicFrame.fromDF(githubCommitsDF, glueContext, "df2")  
parquet_output_path = 's3://' + os.path.join(args['S3_OUTPUT_BUCKET'], args['S3_OUTPUT_KEY_PREFIX'])
datasink4 = glueContext.write_dynamic_frame.from_options(frame = ghsink, connection_type = "s3", connection_options = {"path": parquet_output_path}, format = "parquet", transformation_ctx = "datasink4")
job.commit()

### Upload the Github ETL script to S3
We will be uploading the `github_etl.py` script to S3 now so that Glue can use it to run the PySpark job. You can replace it with your own script if needed. If your code has multiple files, you need to zip those files and upload to S3 instead of uploading a single file like it's being done here.

In [ ]:
script_location = sess.upload_data(path='github_etl.py', bucket=bucket, key_prefix='github/codes')

In [ ]:
# Output location of the data. The input data will be split, transformed, and 
# uploaded to output/train and output/validation
s3_output_bucket = bucket
s3_output_key_prefix = 'github/parquet'

### Calling Glue APIs

Next we'll be creating Glue client via Boto so that we can invoke the `create_job` API of Glue. `create_job` API will create a job definition which can be used to execute your jobs in Glue. The job definition created here is mutable. While creating the job, we are also passing the code location as well as the dependencies location to Glue.

`AllocatedCapacity` parameter controls the hardware resources that Glue will use to execute this job. It is measures in units of `DPU`. For more information on `DPU`, please see [here](https://docs.aws.amazon.com/glue/latest/dg/add-job.html).

In [ ]:
from time import gmtime, strftime
import time

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

job_name = 'github-etl-' + timestamp_prefix
response = glue.create_job(
    Name=job_name,
    Description='PySpark job to extract Github activity comments on repo filtering on PushEvents',
    Role=role, # you can pass your existing AWS Glue role here if you have used Glue before
    ExecutionProperty={
        'MaxConcurrentRuns': 1
    },
    Command={
        'Name': 'glueetl',
        'ScriptLocation': script_location
    },
    DefaultArguments={
        '--job-language': 'python',
    },
    AllocatedCapacity=5,
    Timeout=60,
)
glue_job_name = response['Name']
print(glue_job_name)

The aforementioned job will be executed now by calling `start_job_run` API. This API creates an immutable run/execution corresponding to the job definition created above. We will require the `job_run_id` for the particular job execution to check for status. We'll pass the data and model locations as part of the job execution parameters.

In [ ]:
job_run_id = glue.start_job_run(JobName=job_name,
                                       Arguments = {
                                        '--S3_OUTPUT_BUCKET': s3_output_bucket,
                                        '--S3_OUTPUT_KEY_PREFIX': s3_output_key_prefix
                                       })['JobRunId']
print(job_run_id)

### Checking Glue Job status

Now we will check for the job status to see if it has `succeeded`, `failed` or `stopped`. Once the job is succeeded, we have the transformed data into S3 in Parquet format which we will use to query with Athena and visualize with QuickSight. If the job fails, you can go to AWS Glue console, click on **Jobs** tab on the left, and from the page, click on this particular job and you will be able to find the CloudWatch logs (the link under **Logs**) link for these jobs which can help you to see what exactly went wrong in the job execution.

[glue.get_job_run](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/glue.html#Glue.Client.get_job_run)

In [ ]:
job_run_status = glue.get_job_run(JobName=job_name,RunId=job_run_id)['JobRun']['JobRunState']
while job_run_status not in ('FAILED', 'SUCCEEDED', 'STOPPED'):
    job_run_status = glue.get_job_run(JobName=job_name,RunId=job_run_id)['JobRun']['JobRunState']
    print (job_run_status)
    time.sleep(60)

### Use a [Glue Crawler](https://docs.aws.amazon.com/glue/latest/dg/add-crawler.html) to Discover the transformed data

You can use a crawler to populate the AWS Glue Data Catalog with tables. This is the primary method used by most AWS Glue users. You add a crawler within your Data Catalog to traverse your data stores. The output of the crawler consists of one or more metadata tables that are defined in your Data Catalog. Extract, transform, and load (ETL) jobs that you define in AWS Glue use these metadata tables as sources and targets.

[glue.create_crawler](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/glue.html#Glue.Client.create_crawler)

In [ ]:
parq_crawler_name = 'github_parquet'
parq_crawler_path = 's3://'+bucket+'/github/parquet/'

response = glue.create_crawler(
    Name=parq_crawler_name,
    Role=role,
    DatabaseName=database_name,
    Description='Crawler for the Parquet Github Activity',
    Targets={
        'S3Targets': [
            {
                'Path': parq_crawler_path
            }
        ]
    },
    SchemaChangePolicy={
        'UpdateBehavior': 'UPDATE_IN_DATABASE',
        'DeleteBehavior': 'DEPRECATE_IN_DATABASE'
    }
)

### Start the Glue Crawler

Execute the cell below and browse to the Glue Crawler we created above. From there you will click the `Run Crawler` button to start the crawl of the raw Github activity data.

In [ ]:
print ("Parquet Crawler: https://{0}.console.aws.amazon.com/glue/home?region={0}#crawler:name={1}".format(region, crawler_name))

### Checking Glue crawler status

In [ ]:
crawler_status = glue.get_crawler(Name=parq_crawler_name)['Crawler']['State']
while crawler_status not in ('READY'):
    crawler_status = glue.get_crawler(Name=parq_crawler_name)['Crawler']['State']
    print(crawler_status)
    time.sleep(10)

### View parquet results

In [107]:
cursor.execute('select * from github.parquet limit 100')

df = as_pandas(cursor)
df.head(25)

,id,name,message,url,login,dt_month,dt_day,dt_year,dt_hour
0,8750329709,rizkika zakka palindungan,dfdfdf\n\ndfdf,https://api.github.com/repos/palindungan/miliz...,palindungan,12,13,2018,18
1,8750329712,exo-swf,fr injection on 20181213-185344,https://api.github.com/repos/exodev/integration,exo-swf,12,13,2018,18
2,8750329726,Mark Nelson,fix links,https://api.github.com/repos/oracle/weblogic-k...,markxnelson,12,13,2018,18
3,8750329740,david63,Added Bot protection,https://api.github.com/repos/david63/david63-s...,david63,12,13,2018,18
4,8750329742,exo-swf,ar injection on 20181213-185336,https://api.github.com/repos/exodev/calendar,exo-swf,12,13,2018,18
5,8750329769,pablo,continuing to follow the lecture,https://api.github.com/repos/Pablito14/spring-...,Pablito14,12,13,2018,18
6,8750329769,pablo,"This was supposed to be on branch day1, but to...",https://api.github.com/repos/Pablito14/spring-...,Pablito14,12,13,2018,18
7,8750329769,Pablo Rosales,Merge pull request #4 from Pablito14/day2\n\nDay2,https://api.github.com/repos/Pablito14/spring-...,Pablito14,12,13,2018,18
8,8750329773,Randell Dawson,fixed: removed extra backtick,https://api.github.com/repos/exmelendez/freeCo...,RandellDawson,12,13,2018,18
9,8750329786,Gleb Mineev,after-review adjustments,https://api.github.com/repos/mineevgleb/godot,mineevgleb,12,13,2018,18


### Visualize results

In [105]:
import bokeh
import bokeh.io
bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import HoverTool

import pandas as pd

cursor.execute('select count(message) as msg_cnt, login from github.parquet group by login')

df = as_pandas(cursor)
df.head(10)

#df_parent_objective_value = df_parent[df_parent['FinalObjectiveValue'] > -float('inf')]
# p = figure(plot_width=900, plot_height=400, x_axis_type='datetime',x_axis_label='message count', y_axis_label='login')
# p.circle(source=df, x='msg_cnt', y='login', color='black')

# show(p)

Loading BokehJS ...

,msg_cnt,login
0,4,Kikoenriko
1,1,ranchirino
2,78,Jenniekerf
3,2,sachinbk0411
4,12,hjmjohnson
5,1,fhernand0705
6,4,Clarasticot
7,1,GiseleLarissa
8,3,bochaco
9,13,qunyanm


## Cleanup

In [ ]:
response = glue.delete_crawler(Name=raw_crawler_name)

In [ ]:
response = glue.delete_crawler(Name=parq_crawler_name)

In [ ]:
response = glue.delete_job(JobName=glue_job_name)

In [ ]:
response = glue.delete_database(
    CatalogId = account_id,
    Name = database_name
)